In [1]:
import os
import xml.etree.cElementTree as ET
import pprint
import re
import collections

In [2]:
# Data used - OpenStreep Map(OSM) -- Sunnyvale, CA
# Dataset size = 549MB

DATA = "DATA" 
SAMPLE_DATA = "data_sample.xml"

### Data Wrangling Flow
1. Data Gathering and OSM file Preparation
2. Data Auditing
3. Data Cleaning (Quality and Tidness)
4. Data Analysis with SQL

## Part I -- Audit and Clean the Data

### 1. Data Assessment

In [3]:
tree = ET.parse('data_sample.xml')
root = tree.getroot()

In [4]:
print(root)

<Element 'osm' at 0x10ee989a8>


In [5]:
# First, I need to count unique tags in this XML file 

def count_tags(filename):
    counts = collections.defaultdict(int)
    for line in ET.iterparse(filename, events=("start",)):
        current = line[1].tag
        counts[current] += 1
    return counts

DATA_tags = count_tags(DATA)
SAMPLE_tags = count_tags(SAMPLE_DATA)
pprint.pprint(DATA_tags)
pprint.pprint(SAMPLE_tags)


defaultdict(<class 'int'>,
            {'member': 40975,
             'meta': 1,
             'nd': 3019867,
             'node': 2577605,
             'note': 1,
             'osm': 1,
             'relation': 4361,
             'tag': 1106609,
             'way': 331926})
defaultdict(<class 'int'>,
            {'member': 581,
             'meta': 1,
             'nd': 543,
             'node': 7954,
             'note': 1,
             'osm': 1,
             'relation': 94,
             'tag': 1083,
             'way': 181})


### 2. Patterns in Tags

In [6]:
'''
Second, I need to explore the "k" values for each tags to see if there's any variation or potential issues.
Using the provided 3 regular expressions, I will check the certain patterns in the tags. 

- "lower", for tags that contain only lowercase letters and are valid,
- "lower_colon", for otherwise valid tags with a colon in their names,
- "problemchars", for tags with problematic characters, and
- "other", for other tags that do not fall into the other three categories.
'''


lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

def key_type(element, keys):
    if element.tag == "tag":
        if lower.search(element.attrib['k']):
            keys['lower'] += 1
        elif lower_colon.search(element.attrib['k']):
            keys['lower_colon'] += 1
        elif problemchars.search(element.attrib['k']):
            keys['problemchars'] += 1
        else:
            keys['other'] += 1
    
    return keys


def process_map(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)

    return keys

pprint.pprint(process_map(DATA))

{'lower': 691111, 'lower_colon': 398350, 'other': 17123, 'problemchars': 25}


### 3. Data Quality 

#### Street Names Inconsistence


In [7]:

street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road",
            "Parkway","Way","Circle","Terrace","Real","Mall","Expressway"]
# update the expected list after first runningn(add some street names, such as "Circle", "Terrace", etc.)

def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)


def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")
# looking for tags which specify a street name

def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = collections.defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag in["node", "way", "relation"]:
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    return street_types

st_types = audit(DATA)
pprint.pprint(dict(st_types))



{'0.1': {'Ala 680 PM 0.1'},
 '2': {'Showers Drive STE 2'},
 '3': {'University Dr. Suite 3'},
 '4A': {'Saratoga Avenue Bldg 4A'},
 '7': {'Showers Drive STE 7'},
 '9': {'East Charleston Road APT 9'},
 'A': {'Lane A'},
 'AA': {'Showers Drive BLDG AA'},
 'Alameda': {'The Alameda'},
 'Alley': {'Fountain Alley', 'Jackson Alley'},
 'Ashfield': {'Ashfield'},
 'Ave': {'Bayshore Ave',
         'Blake Ave',
         'Blenheim Ave',
         'Cabrillo Ave',
         'California Ave',
         'E Duane Ave',
         'Forest Ave',
         'Greenbriar Ave',
         'Hollenbeck Ave',
         'Hurlingame Ave',
         'Meridian Ave',
         'N Blaney Ave',
         'Portage Ave',
         'S California Ave',
         'Seaboard Ave',
         'Tehama Ave',
         'The Alameda Ave',
         'University Ave',
         'W Washington Ave',
         'Walsh Ave',
         'Westfield Ave'},
 'Ave.': {'Hamilton Ave.', 'Menalto Ave.', 'Santa Cruz Ave.'},
 'Axis': {'North-South Axis'},
 'B': {'Leghorn S

In [8]:

mapping = { "Ave": "Avenue",
            "Ave.": "Avenue",
            "ave": "Avenue",
            "avenue": "Avenue",
            "Blvd": "Boulevard",
            "Cir": "Circle",
            "Dr": "Drive",
            "Dr.": "Drive",
            "Rd": "Road",
            "St": "Street",
            "St.": "Street",
            "street": "Street",
            "terrace": "Terrace",
            "Hwy": "Highway",
            }

name_list = []

def audit_street_name(osmfile):
    osm_file = open(osmfile,"r")
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag in["node", "way", "relation"] :
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    new_name = update_name(tag.attrib['v'])
                    name_list.append(new_name)
                    
    osm_file.close()
    return name_list


def update_name(name):
    for n in name.split():
        if n in mapping:
            name = name.replace(n, mapping[n])
        else:
            name = name
    return name



st_name = audit_street_name(DATA)
pprint.pprint(st_name)



['Knox Avenue',
 'South Fair Oaks Avenue',
 'West El Camino Real',
 'West El Camino Real',
 'East El Camino Real',
 'East El Camino Real',
 'East El Camino Real',
 'Persian Drive',
 'Halford Avenue',
 'Halford Avenue',
 'West El Camino Real',
 'West El Camino Real',
 'East El Camino Real',
 'East Remington Drive',
 'Hollenbeck Avenue',
 'North 1st Street',
 'North First Street',
 'Leghorn Street',
 'West Evelyn Avenue',
 'Lagunita Drive',
 'Stanford Avenue',
 'Grant Road',
 'South California Avenue',
 'California Avenue',
 'California Avenue',
 'California Avenue',
 'California Avenue',
 'California Avenue',
 'California Avenue',
 'South California Avenue',
 'South California Avenue',
 'El Camino Real',
 'Sunnyvale-Saratoga Road',
 'El Camino Real',
 'Castro Street',
 'Castro Street',
 'Castro Street',
 'Castro Street',
 'Castro Street',
 'Castro Street',
 'Castro Street',
 'Villa Street',
 'Villa Street',
 'Villa Street',
 'East El Camino Real',
 'East El Camino Real',
 'South Taaffe 

 'South Wolfe Road',
 'South Wolfe Road',
 'West El Camino Real',
 'West El Camino Real',
 'West El Camino Real',
 'West El Camino Real',
 'West El Camino Real',
 'Los Altos Avenue',
 'N Wolfe Road',
 'Calabazas Creek Circle',
 'Calabazas Creek Circle',
 'Calabazas Creek Circle',
 'East Duane Avenue',
 'East Duane Avenue',
 'East Duane Avenue',
 'Leghorn Street',
 'Stevens Creek Boulevard',
 'De La Cruz Boulevard',
 'East El Camino Real',
 'Calabazas Creek Circle',
 'Calabazas Creek Circle',
 'Calabazas Creek Circle',
 'Calabazas Creek Circle',
 'Calabazas Creek Circle',
 'Calabazas Creek Circle',
 'Calabazas Creek Circle',
 'Calabazas Creek Circle',
 'Calabazas Creek Circle',
 'Calabazas Creek Circle',
 'Calabazas Creek Circle',
 'Calabazas Creek Circle',
 'Calabazas Creek Circle',
 'Calabazas Creek Circle',
 'Calabazas Creek Circle',
 'Calabazas Creek Circle',
 'Calabazas Creek Circle',
 'Calabazas Creek Circle',
 'Calabazas Creek Circle',
 'Calabazas Creek Circle',
 'Longfellow Way'

 'Alma Street',
 'Alma Street',
 'Coleridge Avenue',
 'Seale Avenue',
 'Tennyson Avenue',
 'Seale Avenue',
 'Coleridge Avenue',
 'Melville Avenue',
 'Seale Avenue',
 'Melville Avenue',
 'Santa Rita Avenue',
 'Bryant Street',
 'Alma Street',
 'Santa Rita Avenue',
 'Alma Street',
 'Alma Street',
 'Seale Avenue',
 'Tennyson Avenue',
 'Alma Street',
 'Alma Street',
 'Rinconada Avenue',
 'Tennyson Avenue',
 'Rinconada Avenue',
 'Coleridge Avenue',
 'Alma Street',
 'Alma Street',
 'Emerson Street',
 'Alma Street',
 'Tennyson Avenue',
 'Emerson Street',
 'Santa Rita Avenue',
 'Emerson Street',
 'Rinconada Avenue',
 'Santa Rita Avenue',
 'Rinconada Avenue',
 'Santa Rita Avenue',
 'Rinconada Avenue',
 'Kellogg Avenue',
 'Cowper Street',
 'Cowper Street',
 'Cowper Street',
 'Cowper Street',
 'Cowper Street',
 'Melville Avenue',
 'Santa Rita Avenue',
 'Melville Avenue',
 'Alma Street',
 'Santa Rita Avenue',
 'Alma Street',
 'Alma Street',
 'Alma Street',
 'Alma Street',
 'Alma Street',
 'Emerson 

 'Varian Way',
 'Clark Way',
 'Clark Way',
 'Varian Way',
 'Clark Way',
 'Clark Way',
 'Clark Way',
 'Varian Way',
 'Clark Way',
 'Varian Way',
 'Clark Way',
 'Varian Way',
 'Clark Way',
 'Varian Way',
 'Varian Way',
 'Clark Way',
 'Varian Way',
 'Clark Way',
 'Varian Way',
 'Clark Way',
 'Varian Way',
 'Clark Way',
 'Varian Way',
 'Clark Way',
 'Varian Way',
 'Clark Way',
 'Varian Way',
 'Clark Way',
 'Varian Way',
 'Clark Way',
 'Varian Way',
 'Clark Way',
 'Varian Way',
 'Clark Way',
 'Clark Way',
 'Varian Way',
 'Clark Way',
 'Clark Way',
 'Varian Way',
 'Clark Way',
 'Varian Way',
 'Varian Way',
 'Varian Way',
 'Varian Way',
 'Varian Way',
 'Clark Way',
 'Varian Way',
 'Varian Way',
 'College Avenue',
 'California Avenue',
 'College Avenue',
 'College Avenue',
 'College Avenue',
 'College Avenue',
 'College Avenue',
 'California Avenue',
 'College Avenue',
 'College Avenue',
 'Stanford Avenue',
 'Amherst Street',
 'Williams Street',
 'Amherst Street',
 'Oberlin Street',
 'Oberlin 

 'Sunnyvale-Saratoga Road',
 'West El Camino Real',
 'West El Camino Real',
 'West El Camino Real',
 'West Maude Avenue',
 'West Maude Avenue',
 'West Maude Avenue',
 'Technology Drive',
 'Santa Cruz Avenue',
 'Santa Cruz Avenue',
 'Santa Cruz Avenue',
 'El Camino Real',
 'El Camino Real',
 'Stevens Creek Boulevard',
 'Stevens Creek Boulevard',
 'Stevens Creek Boulevard',
 'South Bascom Avenue',
 'Lee Drive',
 'Main Street',
 'Main Street',
 'Main Street',
 'Main Street',
 'Main Street',
 'State Street',
 'State Street',
 'Main Street',
 'State Street',
 '1st Street',
 '1st Street',
 'State Street',
 'State Street',
 'Main Street',
 'Main Street',
 'Main Street',
 'Main Street',
 'Main Street',
 'Main Street',
 'Main Street',
 'Main Street',
 'Main Street',
 'West El Camino Real',
 '1st Street',
 '1st Street',
 '2nd Street',
 '2nd Street',
 'Main Street',
 'Main Street',
 'Main Street',
 'Main Street',
 'Main Street',
 'Main Street',
 'Main Street',
 'Main Street',
 'Main Street',
 'Ma

...

 'Tiptoe Lane',
 'De la Farge Drive',
 'Tiptoe Lane',
 'Fallenleaf Lane',
 'Heatherwood Drive',
 'Heatherwood Drive',
 'Fallenleaf Lane',
 'Tiptoe Lane',
 'De la Farge Drive',
 'Heatherwood Drive',
 'Fallenleaf Lane',
 'De la Farge Drive',
 'Tiptoe Lane',
 'Fallenleaf Lane',
 'Heatherwood Drive',
 'Tiptoe Lane',
 'Heatherwood Drive',
 'Fallenleaf Lane',
 'Fallenleaf Lane',
 'Fallenleaf Lane',
 'De la Farge Drive',
 'Tiptoe Lane',
 'Heatherwood Drive',
 'Tiptoe Lane',
 'Heatherwood Drive',
 'Heatherwood Drive',
 'Tiptoe Lane',
 'Tiptoe Lane',
 'Tiptoe Lane',
 'Heatherwood Drive',
 'Tiptoe Lane',
 'Heatherwood Drive',
 'Heatherwood Drive',
 'Elderwood Court',
 'Elderwood Court',
 'Elderwood Court',
 'Elderwood Court',
 'Elderwood Court',
 'Elderwood Court',
 'Elderwood Court',
 'Elderwood Court',
 'Elderwood Court',
 'Elderwood Court',
 'Elderwood Court',
 'Elderwood Court',
 'Elderwood Court',
 'Elderwood Court',
 'Elderwood Court',
 'Elderwood Court',
 'Elderwood Court',
 'De la Farge 

 'La Mar Court',
 'La Mar Drive',
 'La Mar Drive',
 'La Mar Drive',
 'La Mar Drive',
 'La Mar Drive',
 'La Mar Drive',
 'La Mar Drive',
 'La Mar Drive',
 'La Mar Drive',
 'La Mar Drive',
 'La Mar Drive',
 'La Mar Drive',
 'La Mar Drive',
 'La Mar Drive',
 'La Mar Drive',
 'La Mar Drive',
 'La Mar Drive',
 'La Mar Drive',
 'La Mar Drive',
 'La Mar Drive',
 'La Mar Drive',
 'Lindenbrook Lane',
 'Lindenbrook Lane',
 'La Mar Drive',
 'Lindenbrook Lane',
 'La Mar Drive',
 'Lindenbrook Lane',
 'Lindenbrook Lane',
 'La Mar Drive',
 'Lindenbrook Lane',
 'La Mar Drive',
 'Lindenbrook Lane',
 'La Mar Drive',
 'Lindenbrook Lane',
 'La Mar Drive',
 'Lindenbrook Lane',
 'La Mar Drive',
 'Lindenbrook Lane',
 'La Mar Drive',
 'Lindenbrook Lane',
 'Lindenbrook Lane',
 'Lindenbrook Lane',
 'Clifford Drive',
 'Clifford Drive',
 'Clifford Drive',
 'Clifford Drive',
 'Clifford Drive',
 'Clifford Drive',
 'Clifford Drive',
 'John Drive',
 'Clifford Drive',
 'John Drive',
 'John Drive',
 'Clifford Drive',
 

 'South Stelling Road',
 'Tula Lane',
 'Westacres Drive',
 'Paradise Drive',
 'Tonita Way',
 'Bonny Drive',
 'Bonny Drive',
 'Westacres Drive',
 'Tonita Way',
 'Tula Lane',
 'Westacres Drive',
 'Paradise Drive',
 'Tula Lane',
 'Westacres Drive',
 'Bonny Drive',
 'Bonny Drive',
 'Westacres Drive',
 'Paradise Drive',
 'Bonny Drive',
 'Bonny Drive',
 'Tonita Way',
 'Tula Lane',
 'Westacres Drive',
 'Tula Lane',
 'Westacres Drive',
 'Bonny Drive',
 'Tula Lane',
 'South De Anza Boulevard',
 'Westacres Drive',
 'Westacres Drive',
 'Bonny Drive',
 'Bonny Drive',
 'South Stelling Road',
 'Westacres Drive',
 'Westacres Drive',
 'Bonny Drive',
 'Bonny Drive',
 'Scofield Drive',
 'Sunrise Drive',
 'Rodrigues Avenue',
 'Scofield Drive',
 'Sunrise Drive',
 'Sunrise Drive',
 'Rodrigues Avenue',
 'Sunrise Drive',
 'Scofield Drive',
 'Sunrise Drive',
 'Rodrigues Avenue',
 'Scofield Drive',
 'Sunrise Drive',
 'Scofield Drive',
 'Sunrise Drive',
 'Rodrigues Avenue',
 'McClellan Road',
 'Rodrigues Avenue

 'Stierlin Road',
 'Washington Street',
 'Washington Street',
 'Stierlin Road',
 'Fountain Park Lane',
 'Fountain Park Lane',
 'Jackson Street',
 'Jackson Alley',
 'Jackson Street',
 'Fountain Park Lane',
 'Stierlin Road',
 'Elmwood Street',
 'Elmwood Street',
 'Stierlin Road',
 'Jackson Street',
 'Fountain Park Lane',
 'Fountain Park Lane',
 'Jackson Street',
 'Jackson Street',
 'Fountain Park Lane',
 'Fountain Park Lane',
 'West Middlefield Road',
 'West Middlefield Road',
 'West Middlefield Road',
 'North Shoreline Boulevard',
 'North Shoreline Boulevard',
 'North Shoreline Boulevard',
 'North Shoreline Boulevard',
 'North Shoreline Boulevard',
 'North Shoreline Boulevard',
 'North Shoreline Boulevard',
 'North Shoreline Boulevard',
 'North Mary Avenue',
 'North Mary Avenue',
 'North Mary Avenue',
 'North Mary Avenue',
 'North Mary Avenue',
 'North Mary Avenue',
 'North Mary Avenue',
 'North Mary Avenue',
 'East Washington Avenue',
 'East Washington Avenue',
 'East Washington Avenue

 'Rankin Avenue',
 'Rankin Avenue',
 'Rankin Avenue',
 'Rankin Avenue',
 'Rankin Avenue',
 'Rankin Avenue',
 'Rankin Avenue',
 'West Julian Street',
 'West Saint John Street',
 'West Julian',
 'Mary Avenue',
 'Crisanto Avenue',
 'Escuela Avenue',
 'Escuela Avenue',
 'Escuela Avenue',
 'Escuela Avenue',
 'Montecito Avenue',
 'Glen Una Avenue',
 'Glen Una Avenue',
 'Stevens Creek Boulevard',
 'Escuela Avenue',
 'Eunice Avenue',
 'yes',
 'yes',
 'yes',
 'North Monroe Street',
 'Garcia Avenue',
 'Jefferson Drive',
 'Jefferson Drive',
 'Leghorn Street',
 'Willow Street',
 'Andsbury Avenue',
 'Cypress Point Drive',
 'Latham Street',
 'West El Camino Real',
 'Sierra Vista Avenue',
 'Stanford Avenue',
 'Stanford Avenue',
 'Stanford Avenue',
 'Stanford Avenue',
 'Stanford Avenue',
 'Stanford Avenue',
 'Stanford Avenue',
 'Stanford Avenue',
 'Stanford Avenue',
 'Stanford Avenue',
 'Stanford Avenue',
 'Stanford Avenue',
 'Stanford Avenue',
 'Stanford Avenue',
 'Stanford Avenue',
 'Stanford Avenue

#### Postcodes Format

In [9]:
def is_postcode(elem):
    return (elem.attrib['k'] == "addr:postcode")        

def postcode_audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = collections.defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag in ["node","way","relation"]:
            for tag in elem.iter("tag"):
                if is_postcode(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    return street_types

postcode_audit(DATA)

defaultdict(set,
            {'94086': {'94086', 'CA 94086'},
             '94087': {'94087'},
             '95051': {'95051'},
             '95112': {'95112'},
             '94305': {'94305'},
             '94306': {'94306'},
             '95050': {'95050'},
             '94041': {'94041'},
             '94022': {'94022'},
             '94043': {'94043'},
             '94040': {'94040'},
             '95008': {'95008'},
             '94024': {'94024', 'CA 94024'},
             '95070': {'95070'},
             '95117': {'95117'},
             '94303': {'94303'},
             '95125': {'95125'},
             '95110': {'95110', 'CA 95110'},
             '95130': {'95130'},
             '95129': {'95129'},
             '95014': {'95014', 'CA 95014'},
             '94025': {'94025'},
             '94085': {'94085'},
             '95054': {'95054', 'CA 95054'},
             '94063': {'94063'},
             '94301': {'94301'},
             '95126': {'95126'},
             '95134': {'95134'},

## Part II --  Importing to SQL Database

In [10]:
# Note: The schema is stored in a .py file in order to take advantage of the
# int() and float() type coercion functions. Otherwise it could easily stored as
# as JSON or another serialized format.

schema = {
    'node': {
        'type': 'dict',
        'schema': {
            'id': {'required': True, 'type': 'integer', 'coerce': int},
            'lat': {'required': True, 'type': 'float', 'coerce': float},
            'lon': {'required': True, 'type': 'float', 'coerce': float},
            'user': {'required': True, 'type': 'string'},
            'uid': {'required': True, 'type': 'integer', 'coerce': int},
            'version': {'required': True, 'type': 'string'},
            'changeset': {'required': True, 'type': 'integer', 'coerce': int},
            'timestamp': {'required': True, 'type': 'string'}
        }
    },
    'node_tags': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'key': {'required': True, 'type': 'string'},
                'value': {'required': True, 'type': 'string'},
                'type': {'required': True, 'type': 'string'}
            }
        }
    },
    'way': {
        'type': 'dict',
        'schema': {
            'id': {'required': True, 'type': 'integer', 'coerce': int},
            'user': {'required': True, 'type': 'string'},
            'uid': {'required': True, 'type': 'integer', 'coerce': int},
            'version': {'required': True, 'type': 'string'},
            'changeset': {'required': True, 'type': 'integer', 'coerce': int},
            'timestamp': {'required': True, 'type': 'string'}
        }
    },
    'way_nodes': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'node_id': {'required': True, 'type': 'integer', 'coerce': int},
                'position': {'required': True, 'type': 'integer', 'coerce': int}
            }
        }
    },
    'way_tags': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'key': {'required': True, 'type': 'string'},
                'value': {'required': True, 'type': 'string'},
                'type': {'required': True, 'type': 'string'}
            }
        }
    }
}


In [19]:
import csv
import codecs
import re
import xml.etree.cElementTree as ET
from unittest import TestCase

import cerberus
import schema


NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

SCHEMA = schema.schema

NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']


def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""

    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  
    

########################
# my code goes here 
#######################

    # parse the element as node
    if element.tag == 'node':
        
        # parse node child element
        for child_element in element:
            
            node_tag = {}
            
            # if matches problematic re pattern, skip. note: we only have 25 problematic records so it's safe to dump them
            if PROBLEMCHARS.match(child_element.attrib['k']):
                continue
            # if matches lower case with colon re pattern, we store first part of attribte as "type" and second part as "key"
            elif LOWER_COLON.match(child_element.attrib['k']):
                node_tag['type'] = child_element.attrib['k'].split(':',1)[0]
                node_tag['key'] = child_element.attrib['k'].split(':',1)[1]
                node_tag['id'] = element.attrib['id']
                node_tag['value'] = child_element.attrib['v']
                tags.append(node_tag)
            # if no matches, we store regular as "type" and attribute as "key"
            else:
                node_tag['type'] = 'regular'
                node_tag['key'] = child_element.attrib['k']
                node_tag['id'] = element.attrib['id']
                node_tag['value'] = child_element.attrib['v']
                tags.append(node_tag)
        
        # only populates the node_attribs if its value appears in NODE_FIELDS
        for val in element.attrib:
            if val in NODE_FIELDS:
                node_attribs[val] = element.attrib[val]
        
        # return parsed node
        return {'node': node_attribs, 'node_tags': tags}
    
    # parse the element as way    
    elif element.tag == 'way':
        
        node_position = 0
        
        for child_element in element:
            
            way_tag = {}
            way_node = {}
            
            if child_element.tag == 'tag':
                # if matches problematic re pattern, skip. note: we only have 25 problematic records so it's safe to dump them
                if PROBLEMCHARS.match(child_element.attrib['k']):
                    continue
                # if matches lower case with colon re pattern, we store first part of attribte as "type" and second part as "key"
                elif LOWER_COLON.match(child_element.attrib['k']):
                    way_tag['type'] = child_element.attrib['k'].split(':',1)[0]
                    way_tag['key'] = child_element.attrib['k'].split(':',1)[1]
                    way_tag['id'] = element.attrib['id']
                    way_tag['value'] = child_element.attrib['v']
                    tags.append(way_tag)
                # if no matches, we store regular as "type" and attribute as "key"
                else:
                    way_tag['type'] = 'regular'
                    way_tag['key'] = child_element.attrib['k']
                    way_tag['id'] = element.attrib['id']
                    way_tag['value'] = child_element.attrib['v']
                    tags.append(way_tag)
                    
            elif child_element.tag == 'nd':
                way_node['id'] = element.attrib['id']
                way_node['node_id'] = child_element.attrib['ref']
                way_node['position'] = node_position
                node_position = node_position + 1
                way_nodes.append(way_node)
        
        # only populates the way_attribs if its value appears in WAY_FIELDS
        for val in element.attrib:
            if val in WAY_FIELDS:
                way_attribs[val] = element.attrib[val]
                
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}


# ================================================== #
#               Helper Functions                     #
# ================================================== #
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.items())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_strings = (
            "{0}: {1}".format(k, v if isinstance(v, unicode) else ", ".join(v))
            for k, v in errors.items()
        )
        raise cerberus.ValidationError(
            message_string.format(field, "\n".join(error_strings))
        )

# unicode is deprecated in Python 3, this gives me so much trouble...!!! I managed to resolve this problem by changing "unicode" to "str"
class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v if isinstance(v, str) else v) for k, v in row.items()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)


# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w', encoding='utf-8') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w', encoding='utf-8') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w', encoding='utf-8') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w', encoding='utf-8') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w', encoding='utf-8') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])


if __name__ == '__main__':
    process_map(DATA, validate=True)


In [20]:
import sqlite3
import csv 

con = sqlite3.connect("sunnyvale.db")
con.text_factory = str
cur = con.cursor()

# Drop the tables if they already exist....

# cur.execute("DROP TABLE nodes")
# cur.execute("DROP TABLE nodes_tags")
# cur.execute("DROP TABLE ways")
# cur.execute("DROP TABLE ways_nodes")
# cur.execute("DROP TABLE ways_tags")

# create nodes table
cur.execute("CREATE TABLE nodes (id, lat, lon, user, uid, version, changeset, timestamp);")
with open('nodes.csv','rt') as fin:
    dr = csv.DictReader(fin) 
    to_db = [(i['id'], i['lat'], i['lon'], i['user'], i['uid'], i['version'], i['changeset'], i['timestamp']) \
             for i in dr]

cur.executemany("INSERT INTO nodes (id, lat, lon, user, uid, version, changeset, timestamp) \
                VALUES (?, ?, ?, ?, ?, ?, ?, ?);", to_db)
con.commit()

# create nodes_tags table
cur.execute("CREATE TABLE nodes_tags (id, key, value, type);")
# read csv as "text" file
with open('nodes_tags.csv','rt') as fin:
    dr = csv.DictReader(fin) 
    to_db = [(i['id'], i['key'], i['value'], i['type']) for i in dr]

cur.executemany("INSERT INTO nodes_tags (id, key, value, type) VALUES (?, ?, ?, ?);", to_db)
con.commit()

# create ways table
cur.execute("CREATE TABLE ways (id, user, uid, version, changeset, timestamp);")
# read csv as "text" file
with open('ways.csv','rt') as fin:
    dr = csv.DictReader(fin) 
    to_db = [(i['id'], i['user'], i['uid'], i['version'], i['changeset'], i['timestamp']) for i in dr]

cur.executemany("INSERT INTO ways (id, user, uid, version, changeset, timestamp) VALUES (?, ?, ?, ?, ?, ?);", to_db)
con.commit()

# create ways_nodes table
cur.execute("CREATE TABLE ways_nodes (id, node_id, position);")
# read csv as "text" file
with open('ways_nodes.csv','rt') as fin:
    dr = csv.DictReader(fin) 
    to_db = [(i['id'], i['node_id'], i['position']) for i in dr]

cur.executemany("INSERT INTO ways_nodes (id, node_id, position) VALUES (?, ?, ?);", to_db)
con.commit()

# create ways_tags table
cur.execute("CREATE TABLE ways_tags (id, key, value, type);")
# read csv as "text" file
with open('ways_tags.csv','rt') as fin:
    dr = csv.DictReader(fin) 
    to_db = [(i['id'], i['key'], i['value'], i['type']) for i in dr]

cur.executemany("INSERT INTO ways_tags (id, key, value, type) VALUES (?, ?, ?, ?);", to_db)
con.commit()

In [22]:
# Create the SQL database
conn = sqlite3.connect('Sunnyvale.db')
cursor = conn.cursor()
query = '''SELECT * FROM nodes LIMIT 5;'''
cursor.execute(query)
all_rows = cursor.fetchall()
print(all_rows)

[('302863', '37.3769826', '-122.0677726', 'KindredCoda', '14293', '4', '6776161', '2010-12-27T10:41:10Z'), ('302864', '37.3740667', '-122.0679336', 'KindredCoda', '14293', '5', '6776161', '2010-12-27T10:39:22Z'), ('302865', '37.3733004', '-122.0677211', 'KindredCoda', '14293', '4', '6774664', '2010-12-27T07:44:11Z'), ('302866', '37.3725970', '-122.0673831', 'KindredCoda', '14293', '4', '6774664', '2010-12-27T07:43:52Z'), ('302867', '37.3695957', '-122.0655646', 'KindredCoda', '14293', '4', '6774664', '2010-12-27T07:43:25Z')]


##  Data Overview

In [23]:
import os
print('Sunnyvale.osm file is {} MB'.format(os.path.getsize('DATA')/1.0e6))
print('The project.db file is {} MB'.format(os.path.getsize('Sunnyvale.db')/1.0e6))
print('The nodes.csv file is {} MB'.format(os.path.getsize('nodes.csv')/1.0e6))
print('The nodes_tags.csv file is {} MB'.format(os.path.getsize('nodes.csv')/1.0e6))
print('The ways.csv file is {} MB'.format(os.path.getsize('ways.csv')/1.0e6))
print('The ways_tags.csv is {} MB'.format(os.path.getsize('ways_tags.csv')/1.0e6))
print('The ways_nodes.csv is {} MB'.format(os.path.getsize('ways_nodes.csv')/1.0e6)) # Convert from bytes to MB

Sunnyvale.osm file is 575.870831 MB
The project.db file is 398.548992 MB
The nodes.csv file is 218.36976 MB
The nodes_tags.csv file is 218.36976 MB
The ways.csv file is 20.115894 MB
The ways_tags.csv is 33.379364 MB
The ways_nodes.csv is 72.495658 MB
